In [11]:
#Imports
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

In [2]:
data = spark. \
        read. \
        parquet("../../../../spark-nlp/src/test/resources/sentiment.parquet"). \
        limit(10000)
data.cache()
data.count()

10000

In [20]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetectorModel() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = RegexTokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("../../../src/test/resources/lemma-corpus/AntBNC_lemmas_ver_001.txt")
        
sentiment_detector = SentimentDetectorModel() \
    .setInputCols(["lemma", "sentence"]) \
    .setOutputCol("sentiment_score") \
    .setDictPath("../../../src/test/resources/sentiment-corpus/default-sentiment-dict.txt")
    
finisher = Finisher() \
    .setInputCols(["sentiment_score"]) \
    .setOutputCols(["sentiment"])

In [21]:
pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, lemmatizer, sentiment_detector, finisher])
model = pipeline.fit(data)
result = model.transform(data)

In [17]:
result.filter("sentiment != 'positive')".show()

AttributeError: 'function' object has no attribute 'show'